In [ ]:
%pip install --upgrade --quiet langchain langchain-openai langgraph rdflib langchain-community

In [ ]:
from langchain.chains import GraphSparqlQAChain, OntotextGraphDBQAChain
from langchain_community.graphs import RdfGraph, OntotextGraphDBGraph
from langchain_openai import ChatOpenAI
import os

In [ ]:
os.environ["OPENAI_API_KEY"] = ""

In [ ]:

graph_db = OntotextGraphDBGraph(
    query_endpoint="http://localhost:7200/repositories/commissioning",
    query_ontology="CONSTRUCT {?s ?p ?o} WHERE {?s ?p ?o}",
)

In [ ]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [ ]:
from typing import Dict, List, Tuple, Literal
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

In [ ]:
GRAPHDB_SPARQL_GENERATION_TEMPLATE = """
  Write a SPARQL SELECT query for querying a graph database.
  The ontology schema delimited by triple backticks in Turtle format is:
  ```
  {schema}
  ```
  Use only the classes and properties provided in the schema to construct the SPARQL query.
  Do not use any classes or properties that are not explicitly provided in the SPARQL query.
  Include all necessary prefixes.
  Do not include any explanations or apologies in your responses.
  Do not wrap the query in backticks.
  Variable names should match the question.
  Always check the entityType to be sure which entity(ies) the question refers to. For example: activityEntityType 'Activity' refers to activity.
  Do not include any text except the SPARQL query generated.
  The question delimited by triple backticks is:
  ```
  {prompt}
  ```
  """


GRAPHDB_SPARQL_FIX_GENERATION_TEMPLATE = """
  Write a SPARQL SELECT query for querying a graph database, but take into account the feedback or error: {feedback}
  The ontology schema delimited by triple backticks in Turtle format is:
  ```
  {schema}
  ```
  Use only the classes and properties provided in the schema to construct the SPARQL query.
  Do not use any classes or properties that are not explicitly provided in the SPARQL query.
  Include all necessary prefixes.
  Always check the entityType to be sure which entity(ies) the question refers to. For example: activityEntityType 'Activity' refers to activity.
  Do not include any explanations or apologies in your responses.
  Do not wrap the query in backticks.
  Variable names should match the question.
  Do not include any text except the SPARQL query generated.
  The question delimited by triple backticks is:
  ```
  {prompt}
  ```
  """


In [ ]:
class State(TypedDict):
    question: str
    sparql_query: str
    decision: str
    result: str
    raw_result: any
    structured_result: any
    sparql_query_variables: any
    feedback: str
    pass_or_fail: Literal["PASS", "FAIL"]
    chart_type: Literal["BAR", "NONE"]
    sintax_pass_or_fail: Literal["PASS", "FAIL"]

In [ ]:
graph_db = OntotextGraphDBGraph(
    query_endpoint="http://localhost:7200/repositories/testando",
    query_ontology="CONSTRUCT {?s ?p ?o} WHERE {?s ?p ?o}",
)

In [ ]:
from pydantic import BaseModel, ValidationError
from typing import Dict, List

class ChartDataModel(BaseModel):
    data: List[Dict[str, int]]

def is_valid_data(input_data) -> bool:
    try:
        ChartDataModel(data=input_data)
        return True
    except ValidationError:
        return False


def chart_data_evaluator(state: State):
  chart_type = 'BAR' if is_valid_data(state['structured_result']) else 'NONE'
  return {"chart_type": chart_type}

def route_chart(state: State):
  return state["chart_type"]

In [ ]:
from rdflib.plugins.sparql.parser import parseQuery

def sintax_evaluator(state: State):
  try:
    parseQuery(state['sparql_query'])
    return {"sintax_pass_or_fail": 'PASS', "feedback": ''}
  except Exception as e:
    return {"sintax_pass_or_fail": 'FAIL', "feedback": e}


def route_sintax(state: State):
  return state["sintax_pass_or_fail"]

In [ ]:
class Feedback(BaseModel):
    pass_or_fail: Literal["PASS", "FAIL"] = Field(
        description="Decide if the sparql query PASS or FAIL",
    )
    feedback: str = Field(
        description="If the sparql query not pass, provide feedback on how to improve it.",
    )


evaluator = llm.with_structured_output(Feedback)

def data_evaluator(state: State):
  evaluation = evaluator.invoke(
  f"""Avalie se o dado minimamente atende ao que está na pergunta. 
                                
  Question: {state['question']} 

  Data: {state['structured_result']}

  """)
  return {"pass_or_fail": evaluation.pass_or_fail, "feedback": evaluation.feedback}


def route_sparql(state: State):
  return state["pass_or_fail"]

In [ ]:
from turtle import st
from typing import Dict, List, Tuple
from langchain.llms import OpenAI
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from IPython.display import Image, display
from langchain_core.messages import HumanMessage, SystemMessage
from rdflib.plugins.sparql import prepareQuery
import re
import pandas as pd
import matplotlib.pyplot as plt


def generate_sparql_query(state: State):
    print('generate_sparql_query')
    schema = graph_db.schema
    prompt = state["question"]
    formatted_template = GRAPHDB_SPARQL_GENERATION_TEMPLATE.format(schema=schema, prompt=prompt)
    if state.get("feedback"):
        feedback = state["feedback"]
        formatted_template = GRAPHDB_SPARQL_FIX_GENERATION_TEMPLATE.format(schema=schema, prompt=prompt, feedback=feedback)

    result= llm.invoke(formatted_template)
    state['sparql_query'] = result.content
    return state

def run_query_database(state: State):
    print('run_query_database')
    try:
        raw_result = graph_db.query(state['sparql_query'])
        state['raw_result'] = raw_result
        return state
    except:
        sintax_evaluator(state)
    

def process_raw_results(state: State):
    print('process_raw_results')
    import re
    match = re.search(r'SELECT\s+(.+?)\s+WHERE', state['sparql_query'], re.DOTALL | re.IGNORECASE)

    select_part = match.group(0)

    all_variables = re.findall(r'\?\w+', select_part)

    inside_parentheses = re.findall(r'\([^()]*\?(\w+)[^()]*\)', select_part)

    inside_parentheses = {f'?{var}' for var in inside_parentheses}

    returned_variables = [var[1:] for var in all_variables if var not in inside_parentheses]

    structured_result = []


    for entity in state['raw_result']:
        entity_props = {}
        for index, prop_value in enumerate(entity):
            if hasattr(prop_value, "toPython"):
                value = prop_value.toPython()
                if type(prop_value.toPython()).__name__ == 'Decimal':
                    value = float(prop_value.toPython())
            else:
                value = prop_value
                if type(prop_value).__name__ == 'Decimal':
                    value = float(prop_value)
            entity_props[returned_variables[index]] = value
        structured_result.append(entity_props)

    state["structured_result"] = structured_result
    state["sparql_query_variables"] = returned_variables

    print(state["structured_result"])
    return state


def create_bar_chart(state):
    print('create_bar_chart')
    try:
        data = state["structured_result"][0]  
        df = pd.DataFrame(data.items(), columns=["Category", "Value"]) 
        _, ax = plt.subplots(figsize=(8, 5))
        colors = plt.cm.Paired.colors  
        ax.bar(
            df["Category"], 
            df["Value"], 
            color=colors[:len(df)],
            edgecolor="black", 
            linewidth=1.2
        )

        ax.tick_params(axis="x", rotation=0, labelsize=12)
        ax.tick_params(axis="y", labelsize=12)

        for i, v in enumerate(df["Value"]):
            ax.text(i, v + 0.1, v, ha="center", fontsize=10, weight="bold")

        plt.grid(axis="y", linestyle="--", alpha=0.7)
        plt.tight_layout()

        graph_path = "chart.png"
        plt.savefig(graph_path, dpi=300)
    except:
        pass


def create_table(state:State):
    print('create_table')
    try:
        df = pd.DataFrame(state["structured_result"])
        df.to_excel('output.xlsx', sheet_name='Sheet1', index=False)
    except:
        pass    

graph_builder = StateGraph(State)
graph_builder.add_node("generate_sparql_query", generate_sparql_query)
graph_builder.add_node("run_query_database", run_query_database)
graph_builder.add_node("process_raw_results", process_raw_results)
graph_builder.add_node("create_table", create_table)
graph_builder.add_node("create_bar_chart", create_bar_chart)

graph_builder.add_node("chart_data_evaluator", chart_data_evaluator)

graph_builder.add_node("sintax_evaluator", sintax_evaluator)


graph_builder.add_edge(START, "generate_sparql_query")
graph_builder.add_edge("generate_sparql_query", "sintax_evaluator")

graph_builder.add_conditional_edges(
    "sintax_evaluator",
    route_sintax,
    {  
        "PASS": 'run_query_database',
        "FAIL": "generate_sparql_query",
    },
)

graph_builder.add_edge('run_query_database', "process_raw_results")
graph_builder.add_edge('process_raw_results', "create_table")

graph_builder.add_edge('create_table', "chart_data_evaluator")


graph_builder.add_conditional_edges(
    "chart_data_evaluator",
    route_chart,
    {  
        "BAR": 'create_bar_chart',
        "NONE": END,
    },
)
graph_builder.add_edge('create_bar_chart', END)


graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))


In [ ]:
validation = [
  {
    "question": "What is the status of the digital process named 'Digital Process 1'?",
    "sparql_query_expected": "PREFIX : <http://proton.semanticweb.org/protonsys#>\nPREFIX comm: <http://example.com/certi/commissioning#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT ?status\nWHERE {\n  ?digitalProcess rdf:type comm:DigitalProcessInstance ;\n                  comm:dpiName \"Digital Process 1\" ;\n                  comm:dpiStatus ?status .\n}"
  },
  {
    "question": "What is the type of the element named 'EL001'?",
    "sparql_query_expected": "PREFIX : <http://proton.semanticweb.org/protonsys#>\nPREFIX comm: <http://example.com/certi/commissioning#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT ?elementType WHERE {\n  ?element rdf:type comm:Element ;\n           comm:elementName \"EL001\" ;\n           comm:elementType ?elementType .\n}"
  },
  {
    "question": "Which groups are responsible for the activities 'Installation - PART 1' and 'Verification - PART 2'?",
    "sparql_query_expected": "PREFIX : <http://proton.semanticweb.org/protonsys#>\nPREFIX comm: <http://example.com/certi/commissioning#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT ?activityName ?responsibleGroup WHERE {\n  ?activity rdf:type comm:Activity ;\n            comm:activityName ?activityName ;\n            comm:activityResponsibleGroup ?responsibleGroup ;\n            comm:activityEntityType \"Activity\" .\n  FILTER (?activityName IN (\"Installation - PART 1\", \"Verification - PART 2\"))\n}"
  },
  {
    "question": "What are the public IDs of the pendencies that are completed?",
    "sparql_query_expected": "PREFIX : <http://proton.semanticweb.org/protonsys#>\nPREFIX comm: <http://example.com/certi/commissioning#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT ?pendencyPublicId\nWHERE {\n  ?pendency a comm:Pendency ;\n            comm:pendencyStatus \"COMPLETED\" ;\n            comm:pendencyPublicId ?pendencyPublicId .\n}"
  },
  {
    "question": "What are the names of the tasks executed by 'Actor A' and 'Actor D'?",
    "sparql_query_expected": "PREFIX : <http://proton.semanticweb.org/protonsys#>\nPREFIX comm: <http://example.com/certi/commissioning#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT ?taskName\nWHERE {\n  ?task a comm:Task ;\n        comm:taskExecutingActor ?actor ;\n        comm:taskName ?taskName .\n  FILTER (?actor = \"Actor A\" || ?actor = \"Actor D\")\n}"
  },
  {
    "question": "What are the names, descriptions, and types of the elements associated with 'Digital Process 1'?",
    "sparql_query_expected": "PREFIX : <http://proton.semanticweb.org/protonsys#>\nPREFIX comm: <http://example.com/certi/commissioning#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT ?elementName ?elementDescription ?elementType\nWHERE {\n  comm:DigitalProcessInstance_1 comm:hasElementAssociated ?element .\n  ?element comm:elementName ?elementName ;\n           comm:elementDescription ?elementDescription ;\n           comm:elementType ?elementType ;\n           comm:elementEntityType \"Element\" .\n}"
  },
  {
    "question": "What is the description of the step succeeding the step 'Installation'?",
    "sparql_query_expected": "PREFIX : <http://proton.semanticweb.org/protonsys#>\nPREFIX comm: <http://example.com/certi/commissioning#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT ?stepDescription WHERE {\n  ?step a comm:Step ;\n        comm:stepName \"Installation\" ;\n        comm:hasStepSuccessor ?successorStep .\n  ?successorStep comm:stepDescription ?stepDescription .\n}"
  },
  {
    "question": "In which steps is the element 'EL002' present?",
    "sparql_query_expected": "PREFIX : <http://proton.semanticweb.org/protonsys#>\nPREFIX comm: <http://example.com/certi/commissioning#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT ?stepName\nWHERE {\n  ?step a comm:Step ;\n        comm:hasDigitalProcessInstance ?dpi .\n  ?dpi comm:hasElementAssociated ?element .\n  ?element comm:elementName \"EL002\" .\n  ?step comm:stepName ?stepName .\n}"
  },
  {
    "question": "What is the status of the pendency associated with Task 6?",
    "sparql_query_expected": "PREFIX : <http://proton.semanticweb.org/protonsys#>\nPREFIX comm: <http://example.com/certi/commissioning#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT ?pendencyStatus WHERE {\n  ?pendency comm:pendencyHasTaskAssociated comm:Task_6 ;\n            comm:pendencyStatus ?pendencyStatus .\n}"
  },
  {
    "question": "What is the status and the creator of the status of Task 2?",
    "sparql_query_expected": "PREFIX : <http://proton.semanticweb.org/protonsys#>\nPREFIX comm: <http://example.com/certi/commissioning#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT ?statusContent ?statusCreator\nWHERE {\n  comm:Task_2 comm:hasStatus ?status .\n  ?status comm:statusContent ?statusContent ;\n          comm:statusCreator ?statusCreator .\n}"
  },
  {
    "question": "List all activities of 'Digital Process 1'. Return name, description, and status.",
    "sparql_query_expected": "PREFIX : <http://proton.semanticweb.org/protonsys#>\nPREFIX comm: <http://example.com/certi/commissioning#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT ?name ?description ?status\nWHERE {\n  ?activity a comm:Activity ;\n            comm:activityName ?name ;\n            comm:activityDescription ?description ;\n            comm:activityStatus ?status ;\n            comm:hasElementAssociated ?element .\n  comm:DigitalProcessInstance_1 comm:hasElementAssociated ?element .\n}"
  },
  {
    "question": "List all tasks of 'Digital Process 1'. Return name, description",
    "sparql_query_expected": "PREFIX : <http://proton.semanticweb.org/protonsys#>\nPREFIX comm: <http://example.com/certi/commissioning#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT ?name ?description\nWHERE {\n  ?task a comm:Task ;\n        comm:taskName ?name ;\n        comm:taskDescription ?description ;\n        comm:hasElement ?element .\n  comm:DigitalProcessInstance_1 comm:hasElementAssociated ?element .\n}"
  },
  {
    "question": "List all tasks that are within the step 'Installation'. Return name, description",
    "sparql_query_expected": "PREFIX : <http://proton.semanticweb.org/protonsys#>\nPREFIX comm: <http://example.com/certi/commissioning#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT ?name ?description\nWHERE {\n  ?task a comm:Task ;\n        comm:taskName ?name ;\n        comm:taskDescription ?description ;\n        comm:hasActivityAssociated ?activity .\n  ?activity comm:hasStepAssociated comm:Step_1 .\n}"
  },
  {
    "question": "List all activities that have a status of in progress or completed. Return name, description and status.",
    "sparql_query_expected": "PREFIX : <http://proton.semanticweb.org/protonsys#>\nPREFIX comm: <http://example.com/certi/commissioning#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT ?name ?description ?status\nWHERE {\n  ?activity a comm:Activity ;\n            comm:activityEntityType \"Activity\" ;\n            comm:activityName ?name ;\n            comm:activityDescription ?description ;\n            comm:activityStatus ?status .\n  FILTER(?status IN (\"IN_PROGRESS\", \"COMPLETED\"))\n}"
  },
  {
    "question": "List all pendencies of the step 'Verification'. Return name, description and status.",
    "sparql_query_expected": "PREFIX : <http://proton.semanticweb.org/protonsys#>\nPREFIX comm: <http://example.com/certi/commissioning#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT ?pendencyName ?pendencyDescription ?pendencyStatus\nWHERE {\n  ?step a comm:Step ;\n        comm:stepName \"Verification\" ;\n        comm:hasDigitalProcessInstance ?dpi .\n  \n  ?pendency a comm:Pendency ;\n            comm:pendencyHasTaskAssociated ?task ;\n            comm:pendencyName ?pendencyName ;\n            comm:pendencyDescription ?pendencyDescription ;\n            comm:pendencyStatus ?pendencyStatus .\n  \n  ?task comm:hasActivityAssociated ?activity .\n  \n  ?activity comm:hasStepAssociated ?step .\n}"
  },
  {
    "question": "How many activities and tasks exist in the step 'Verification'?",
    "sparql_query_expected": "PREFIX : <http://proton.semanticweb.org/protonsys#>\nPREFIX comm: <http://example.com/certi/commissioning#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT (COUNT(DISTINCT ?activity) AS ?activityCount) (COUNT(DISTINCT ?task) AS ?taskCount)\nWHERE {\n  comm:Step_2 comm:stepName \"Verification\" ;\n                 comm:hasDigitalProcessInstance ?dpi .\n\n  ?activity comm:hasStepAssociated comm:Step_2 ;\n            comm:activityEntityType \"Activity\" .\n\n  ?task comm:hasActivityAssociated ?activity ;\n        comm:taskEntityType \"Task\" .\n}"
  },
  {
    "question": "How many tasks are completed an in progress?",
    "sparql_query_expected": "PREFIX : <http://proton.semanticweb.org/protonsys#>\nPREFIX comm: <http://example.com/certi/commissioning#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT (COUNT(?taskCompleted) AS ?completedCount) (COUNT(?taskInProgress) AS ?inProgressCount)\nWHERE {\n  {\n    ?taskCompleted a comm:Task ;\n                   comm:hasStatus ?statusCompleted .\n    ?statusCompleted comm:statusContent \"COMPLETED\" .\n  }\n  UNION\n  {\n    ?taskInProgress a comm:Task ;\n                    comm:hasStatus ?statusInProgress .\n    ?statusInProgress comm:statusContent \"IN_PROGRESS\" .\n  }\n}"
  },
  {
    "question": "How many activities have not started?",
    "sparql_query_expected": "PREFIX : <http://proton.semanticweb.org/protonsys#>\nPREFIX comm: <http://example.com/certi/commissioning#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT (COUNT(?activity) AS ?notStartedActivitiesCount)\nWHERE {\n  ?activity a comm:Activity ;\n            comm:activityStatus \"NOT_STARTED\" ;\n            comm:activityEntityType \"Activity\" .\n}"
  },
  {
    "question": "How many pendencies exist in 'Digital Process 1'?",
    "sparql_query_expected": "PREFIX : <http://proton.semanticweb.org/protonsys#>\nPREFIX comm: <http://example.com/certi/commissioning#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT (COUNT(?pendency) AS ?pendencyCount)\nWHERE {\n  ?pendency a comm:Pendency ;\n            comm:pendencyEntityType \"Pendency\" ;\n            comm:pendencyHasElementAssociated ?element .\n  comm:DigitalProcessInstance_1 comm:hasElementAssociated ?element ;\n                                    comm:dpiName \"Digital Process 1\" .\n}"
  },
  {
    "question": "What is the percentage of completed, in-progress and not started tasks?",
    "sparql_query_expected": "PREFIX : <http://proton.semanticweb.org/protonsys#>\nPREFIX comm: <http://example.com/certi/commissioning#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT ?status (COUNT(?task) AS ?count) (COUNT(?task) * 100 / ?total AS ?percentage)\nWHERE {\n  {\n    SELECT (COUNT(?task) AS ?total)\n    WHERE {\n      ?task rdf:type comm:Task ;\n            comm:taskEntityType \"Task\" .\n    }\n  }\n  ?task rdf:type comm:Task ;\n        comm:taskEntityType \"Task\" ;\n        comm:hasStatus ?statusNode .\n  ?statusNode comm:statusContent ?status .\n}\nGROUP BY ?status ?total"
  }
]


In [ ]:
from ragas.metrics import LLMSQLEquivalence
from ragas.dataset_schema import SingleTurnSample
from ragas.llms import LangchainLLMWrapper
evaluator_llm = LangchainLLMWrapper(llm)

from _sparql_semantic_equivalence import LLMSPARQLEquivalence

In [ ]:
questions = {
  "LEVEL 1": [
    "What is the status of the digital process named 'Digital Process 1'?",
    "What is the type of the element named 'EL001'?",
    "Which groups are responsible for the activities 'Installation - PART 1' and 'Verification - PART 2'?",
    "What are the public IDs of the pendencies that are completed?",
    "What are the names of the tasks executed by 'Actor A' and 'Actor D'?"
  ],
  "LEVEL 2": [
    "What are the names, descriptions, and types of the elements associated with 'Digital Process 1'?",
    "What is the description of the step succeeding the step 'Installation'?",
    "In which steps is the element 'EL002' present?",
    "What is the status of the pendency associated with Task 6?",
    "What is the status and the creator of the status of Task 2?"
  ],
  "LEVEL 3": [
    "List all activities of 'Digital Process 1'. Return name, description, and status.",
    "List all tasks of 'Digital Process 1'. Return name, description",
    "List all tasks that are within the step 'Installation'. Return name, description",
    "List all activities that have a status of in progress or completed. Return name, description and status.",
    "List all pendencies of the step 'Verification'. Return name, description and status."
  ],
  "LEVEL 4": [
    "How many activities and tasks exist in the step 'Verification'?",
    "How many tasks are completed an in progress?",
    "How many activities have not started?",
    "How many pendencies exist in 'Digital Process 1'?",
    "What is the percentage of completed, in-progress and not started tasks?"
  ]
}


responses = []

count = 0
for key in questions:
  r = []
  v = []
  for question in questions[key]:
    print(question)
    state= graph.invoke({'question': question}, {"recursion_limit": 100})
    r.append(state)
    sample = SingleTurnSample(
      response=state['sparql_query'],
      reference=validation[count]['sparql_query_expected'],
      reference_contexts=[
          graph_db.get_schema
      ]
    )
    scorer = LLMSPARQLEquivalence()
    scorer.llm = evaluator_llm
    validation[count]['pass'] = await scorer.single_turn_ascore(sample)
    count +=1

  
  responses.append({key:r})



In [ ]:
state= graph.invoke({'question': "What are the public IDs of the pendencies that are completed?"}, {"recursion_limit": 100})